In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, ResFlow, NVPFlow, phase, Normal, floatX
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

np.random.seed(1234)

In [2]:
n = 32
s1 = 0.01
m1 = 0.
s0 = 0.5
dim = [1,2]

params = []
params.append(np.random.normal(size=dim, scale=s0))
for i in range(n-1):
    new = params[i] + np.random.normal(loc=m1, scale=s1, size=dim)
    params.append(new)
params = np.array(params)

PWalk = NormalRW(dim[0]*dim[1], sigma=s1, mu=m1, sigma0=s0)

In [3]:
params.shape

(32, 1, 2)

In [4]:
def autoregr(X, param):
    d = param[:,:dim[-1]-1]
    X = np.matmul(X, d) + param[:,-1][np.newaxis] + np.random.normal(size=[1,dim[-1]-1], scale=0.1)
    return X

def autoregr_tf(X, param):
    d = param[:,:dim[-1]-1]
    X = tf.matmul(X, d) + param[:,-1][np.newaxis]
    return X

In [5]:
xs = [np.random.normal(size=[1,dim[0]])]
for i in range(n-1):
    xs.append(autoregr(xs[i], params[i]))
xs = np.concatenate(xs)[np.newaxis].astype(floatX)

In [6]:
xs[:,:6]

array([[[ 1.03380073],
        [-0.5918482 ],
        [-0.54354248],
        [-0.83599905],
        [-0.78511876],
        [-0.74559205]]])

In [7]:
def create_step_flow(name=None, prev_flow_output=None, init_sigma=1.):
    with tf.variable_scope(name, reuse=False):
        
        if prev_flow_output is not None:
            aux_vars = prev_flow_output
        else:
            aux_vars = None
            
        step_flow = DFlow([NVPFlow(dim=dim[0]*dim[1], name='res{}'.format(i), aux_vars=aux_vars)\
                           for i in range(4)], init_sigma=init_sigma)
    return step_flow

In [8]:
flows = [create_step_flow('step_flow0', init_sigma=s0)]
outputs = [flows[-1].output]
for i in range(n-1):
    new = create_step_flow('step_flow' + str(i+1), outputs[-1], init_sigma=s1)
    flows.append(new)
    outputs.append(outputs[-1] + new.output)

In [9]:
outputs = tf.concat(outputs, axis=0)[tf.newaxis]

In [10]:
outputs

<tf.Tensor 'strided_slice:0' shape=(1, 32, 2) dtype=float64>

In [11]:
prior = PWalk.logdens(outputs)

In [12]:
prior

<tf.Tensor 'add_33:0' shape=() dtype=float64>

In [13]:
def create_loglik():
    obs_d = Normal(dim=None, sigma=0.1, mu=0)
    out = tf.reshape(outputs, [n, dim[0], dim[1]])
    
    ll = 0
    for i in range(n-1):
        pred = xs[0,i+1] - autoregr_tf(xs[0,i][np.newaxis], out[i])
        ll += obs_d.logdens(pred)
    return ll

def create_loglik2():
    obs_d = Normal(dim=None, sigma=0.1, mu=0)
    out = tf.reshape(outputs, [n, dim[0], dim[1]])
    
    def step1(current):
        X = current[0][np.newaxis]
        param = current[1]
        d = param[:,:dim[-1]-1]
        X = tf.matmul(X, d)[0] + param[:,-1]
        return X
    
    ar = tf.map_fn(step1, (xs[0][:-1], out[:-1]), dtype=tf.float64)
    return obs_d.logdens(ar - xs[0][1:])

In [14]:
logl = create_loglik2()

In [15]:
ent = tf.reduce_sum([flow.logdens for flow in flows])

In [16]:
ent

<tf.Tensor 'Sum_3:0' shape=() dtype=float64>

In [17]:
loss = -logl - prior + ent
loss /= n*dim[0]*dim[1]

In [18]:
opt = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [19]:
sess = tf.InteractiveSession()

In [20]:
tf.global_variables_initializer().run()

In [21]:
prior.eval(), ent.eval(), logl.eval()

(131.51456971601888, 188.75074173643128, -229.86590924804472)

In [22]:
loss.eval()

67.8885851086779

In [54]:
%%time
for _ in range(500):
    for _ in range(200):
        l, _ = sess.run([loss, opt], {phase:True})
    print(l)

-0.25770675307390345
-0.2509422622818698
-0.2147738337154368
-0.25353378954868067
-0.27313545591427557
-0.2262374754620584
-0.21685699200991815
-0.25411235080052963
-0.2504318101216647
-0.2608039667799167
-0.25052559874028546
-0.269110518369406
-0.22928589873691152
-0.26491349694489275
-0.22873667582759527
-0.24023828101289046
-0.26874836890208353
-0.22542274461348288
-0.25898749045288394
-0.2602867873560064
-0.2501578254960042
-0.252626324541922
-0.26284046108958226
-0.24882689040801775
-0.25424802147379566
-0.23836378878018794
-0.2594049405848251
-0.26719992197088205
-0.24176872266986082
-0.24789297735908988
-0.2485685236429478
-0.2605199093430022
-0.265043846652814
-0.2570096553464283
-0.21171150999019295
-0.2803806493630496
-0.22415835029883935
-0.25760776582076295
-0.22942389756380432
-0.24279511229715656
-0.2486659339662185
-0.241473401596215
-0.23467947000785605
-0.2869207247491876
-0.23493812620619625
-0.2703086698658761
-0.251617651320331
-0.25963508172456695
-0.30478695813882

KeyboardInterrupt: 

In [24]:
out = tf.reshape(outputs, [n, dim[0], dim[1]])

In [ ]:
samples = []
for _ in range(1000):
    s = out.eval()
    samples.append(s)
samples = np.array(samples)

In [ ]:
samples.shape

In [ ]:
sns.kdeplot(samples[:,1,0,1], samples[:,31,0,1])
plt.show()

In [ ]:
t = 18
params[t]

In [ ]:
np.mean(samples[:,t], axis=0)